In [154]:
import requests
import numpy as np
import re
from gensim.models import Word2Vec
from nltk import word_tokenize

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Acquiring User Likes


### Attempting through Twitter API -- See 1.2 for code that was finally used

In [152]:
### Acquired from Twitter using API key and API secret key ###
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAImyAwEAAAAAyWB9oAfMqhJG66%2BeJHuUEi5wsXM%3DKOzTDPgjqoXjzERO1JmoAJfVIqjnaPdD7wUhayNjESzeytty2J' # Bearer Token


In [19]:
def get_liked_tweets(handle):
    '''
    Takes Twitter handle and extracts and parses liked tweets.
    
    '''
    all_tweets = []
    url = 'https://api.twitter.com/1.1/favorites/list.json?screen_name={}'.format(handle)
    headers = {
        'Authorization': 'Bearer {}'.format(bearer_token),
            }
    first_url_params = {
        'count': 200
            }
    first_response = requests.get(url, headers=headers, params=first_url_params)
    first_initial_split = first_response.text.split('"text":"')
    first_t = len(first_initial_split)
    for i in range(1,first_t):  
        all_tweets.append(first_initial_split[i].split(',"truncated":')[0])
    
    ### Regex to extract the id of the last tweet to use as pagination ###      
    regsearch = re.search('(?<=(,"id":))[\d]*', first_initial_split[first_t-2])
    tweet_id = int(regsearch.group(0))
    ###
    
    ### Looping process to manually paginate through tweets ###
    page = 0
    while page < 6:
        url_params = {
        'count': 200,
        'max_id': tweet_id-1
            }
        response = requests.get(url, headers=headers, params=url_params)
        initial_split = response.text.split('"text":"')
        t = len(initial_split)
        print("attempting another one")
        ### Regex to extract the id of the last tweet to use as pagination ###      
        regsearch = re.search('(?<=(,"id":))[\d]*', initial_split[t-2])
        tweet_id = int(regsearch.group(0))
        ###
        for i in range(1, t):
            all_tweets.append(initial_split[i].split(',"truncated":')[0])
        page +=1

    return all_tweets

In [20]:
# twitter_handle = input("What is your Twitter Handle?")    ### Commented so I don't have to re-type the handle

In [21]:
liked_tweets = get_liked_tweets(twitter_handle)

attempting another one
attempting another one
attempting another one
attempting another one
attempting another one
attempting another one


In [31]:
data = liked_tweets.map(word_tokenize)

NameError: name 'liked_tweets' is not defined

In [78]:
model = Word2Vec([liked_tweets],size=100,window=5,min_count=3,workers=4)

In [79]:
model.train([liked_tweets],total_examples=model.corpus_count,epochs=10)

(100000, 3779140)

In [146]:
wv = model.wv

In [145]:
wv.most_similar('harry')

KeyError: "word 'harry' not in vocabulary"

### Attempting through Twint

In [1]:
import twint
import nest_asyncio
import pandas as pd
from gensim.models import Word2Vec
from nltk import word_tokenize
nest_asyncio.apply()

In [137]:
# Configure
c = twint.Config()
c.Username = 'Brit_Michele_'
c.Pandas = True
c.Limit = 10000
twint.run.Favorites(c)

likes_df = twint.storage.panda.Tweets_df

1197256751126663169 2019-11-20 15:53:48 EST <BernieSanders> I want to thank @ArianaGrande for not only being a wonderful entertainer, but also for being such an outstanding advocate for social justice. We must all be prepared – like Ariana has shown – to fight for everyone who is struggling. It was great to meet her in Atlanta last night. pic.twitter.com/gZTPSLLywX
1197254237685768198 2019-11-20 15:43:48 EST <madiR0see> @Brit_Michele_ why I am NOT attending woods road thanksgiving this year https://twitter.com/kaitlynmidgett/status/1197253134441222145 …
1197236516466155522 2019-11-20 14:33:23 EST <gaialect> if astrology isn’t real then why do ALL leos walk and talk as if they have an audience at all times and are starring in their own movie
1197143051656945666 2019-11-20 08:22:00 EST <laurynvalia> if you honk your horn at me to turn right on a red light, guess what buddy? we’re staying the entire light now. enjoy 😊
1197059896815968261 2019-11-20 02:51:34 EST <todderic_> what the hell i

1194767840500281344 2019-11-13 19:03:45 EST <kaitlynmidgett> I might be single and also depressed but at least I have my own snack drawer at work
1194763197783277569 2019-11-13 18:45:18 EST <DavidDobrik> Better luck next year boys pic.twitter.com/6ye0KiwJdo
1194741018521997312 2019-11-13 17:17:10 EST <bitchrepIies> Dating someone who pushes you to do better is so important
1194729089858301952 2019-11-13 16:29:46 EST <ImNotScottySire> i just watched the first episode of hannah montana for the first time and i teared up when lily and miley became friends again
1194706903768584192 2019-11-13 15:01:37 EST <JbKnockout> pixar please do these again why did you stop doing these pic.twitter.com/IgXbRvkp5N
1194689676646535170 2019-11-13 13:53:09 EST <astroashleigh> this made me feel really sad... i want to tell him he’s a good boy https://twitter.com/rottenmolar/status/1194647978914566147 …
1194644627992850432 2019-11-13 10:54:09 EST <emmymhartman> I literally have to buy 9 outfits Harry Styles 

1193585659031818242 2019-11-10 12:46:11 EST <petexzen> My hair is stupid and unacceptable in a world without MySpace pic.twitter.com/RN5K0FAn0g
1193574862981386240 2019-11-10 12:03:17 EST <perolikewyd> if i followed you i think you’re CUTE AS FUCK
1193366463295508481 2019-11-09 22:15:11 EST <amanda_c_rae> Someone take me on a “let’s grab a hot chocolate and drive around to look at pretty Christmas lights” kind of date pls
1193288672998645760 2019-11-09 17:06:04 EST <katelyn_nevarez> anxiety level: andre’s grandma from victorious
1193233152430542851 2019-11-09 13:25:27 EST <DrizzyJayyyy> when you’re soft as fuck but you had enough pic.twitter.com/DRCZMjwrhI
1193230138315460608 2019-11-09 13:13:28 EST <OregonProgress> Cinderblock doing a full U turn before pouncing. pic.twitter.com/KLCd8xxpVt
1193206437452881932 2019-11-09 11:39:18 EST <Harry_Styles> FINE LINE LIVE AT THE FORUM IS SOLD OUT!   That was wild, thank you so much.  I can’t wait to see you December 13th.  More coming soon.  Lo

1191413212991967233 2019-11-04 12:53:40 EST <kaitlynmidgett> Brittany, it’s monday
1191400063500771329 2019-11-04 12:01:24 EST <Harry_Styles> FINE LINE . THE ALBUM . DEC 13 pic.twitter.com/ARzqYds8Vn
1191322346910560256 2019-11-04 06:52:35 EST <KeannaSosa> Ignoring red flags because I see potential/good in people... https://twitter.com/jasminebell98/status/1189202525389578241 …
1191124084530266112 2019-11-03 17:44:46 EST <spicebae_> tough pill to swallow but some of y’all want closure bc you want them to break down, tell you how sorry they are for doing what they did & how they’d do anything to take it back when in reality they don’t care, they’re not sorry & they just want to do wtf they want to do w/o you.
1191120444780404737 2019-11-03 17:30:18 EST <jodidercolee> Seasonal depression coming to team up with my regular depression pic.twitter.com/fs0g5YVVhV
1191022408070766593 2019-11-03 11:00:44 EST <ImNotScottySire> ok how the fuck do you take eyeliner off are my eyes just permanently

1188350085203222528 2019-10-27 03:01:53 EST <repromantics> 1989 WAS LITERALLY THAT ALBUM. SHE USED :  a retractable pen as sound for Blank Space  a recording tape for I Know Places  her own heartbeats on Wildest Dreams  sampled water droplets on Clean  NAME A MORE GENIUS ARTIST
1188307226705956864 2019-10-27 00:11:35 EST <hurttfuI> yall ever just miss your own energy?? like shit wtf happened to me
1188274791968595968 2019-10-26 22:02:42 EST <GoodishIntent> When my son Seth was little, he would beg his brother to put him in this particular pillow case and swing him around.   I know it’s insane and the only reason I’m telling you now is because they are grown up and I can’t lose custody of them pic.twitter.com/OCTe1vyjkY
1188257541312016384 2019-10-26 20:54:09 EST <popligion> Most embarrassing moments in pop culture: a thread
1188179900164952071 2019-10-26 15:45:38 EST <Iowkeymood> your girl coming to get kisses and cuddles after she just got done yelling at you for no reason pic.twitter

...

1072679489791057926 2018-12-11 21:28:14 EST <dog_feelings> the human seems distracted. so i put my paw on their knee. and now they know. that whatever it is. we’re in it together
1072655745899028486 2018-12-11 19:53:53 EST <astroIogee> my depression on it’s way as soon as the sun starts to set pic.twitter.com/ItDTVNi7wP
1072654770727542785 2018-12-11 19:50:00 EST <ColleenB123> my little love pic.twitter.com/uzfDCGQGp2
1072627149302104066 2018-12-11 18:00:15 EST <JoeyKing> The Kissing Booth is Netflix’s most rewatched movie of 2018 WOW! That’s incredible. Thank you, with all my heart for loving this movie the way I do. You all have shown TKB so much support and it means more than I can express. I love you guys so very much❤️ #ImaCry
1072532302088032256 2018-12-11 11:43:22 EST <kyreetyree> If jersey could have warm weather all year around, we would be unstoppable.
1072514041300566016 2018-12-11 10:30:48 EST <BrittNic18> Vent to me about your problems  But dont make that our only form of 

CRITICAL:root:twint.feed:Mobile:list index out of range


1071573029820362752 2018-12-08 20:11:33 EST <milkstrology> Pooh bear reminds me of Taurus, Leo, and Cancer who all have this strong desire to be comfortable and make anything feel "homey" or familiar no matter what the circumstance. Also, who they are when they are most vulnerable. A kind naive bb, who just finna snacccc. pic.twitter.com/OsKI7N54U9
1071520791173447680 2018-12-08 16:43:59 EST <kaitlynmidgett> fake
1071505468156194822 2018-12-08 15:43:05 EST <lenasantina> and y’all boys are predators praying on young girls. But we much rather blame a freshman for being naive than blame a Senior for being a predator. https://twitter.com/juiceboy_jon/status/1071170390033084422 …
1071474812797767680 2018-12-08 13:41:17 EST <toriaivey> Respect your exes. You shared time. You shared energy. Have respect for them. Don’t talk about them like you didn’t choose to be in their presence daily.
1071276965921996806 2018-12-08 00:35:06 EST <chelseychante> If a parent walks out of their child’s life, i

CRITICAL:root:twint.feed:Mobile:list index out of range


1071242390009110529 2018-12-07 22:17:43 EST <lilireinhart> I’m realizing that having a child gives you the perfect excuse to not leave your house at night or go to things.   Not saying I’m ready to have a baby.  But honestly my future kid will save my ass from so many events, god bless


In [138]:
data = likes_df['tweet']

In [139]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /Users/reuben/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [140]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [141]:
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/reuben/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [142]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in stopwords]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [143]:
import random
text_data = []
for line in data:
    tokens = prepare_text_for_lda(line)
    if random.random() > .99:
        print(tokens)
        text_data.append(tokens)

['coworker', 'loosely', 'translate', 'accident', 'finally', 'place', 'split', 'money', 'please']
['hire', 'friend']
['leaving', 'supermarket', 'halloween']
['explain', 'things']
['bitch', 'bitch', 'sensitive', 'bitch']
['anywhere', 'pic.twitter.com/jg3ayhs2k9']
['canela', 'enjoy', 'summer', 'pic.twitter.com/egztopntuf']
['watching', 'steve', 'dustin', 'reunion', 'scene', 'least', 'pic.twitter.com/l3b7uspez8']
['talk', 'little', 'jairus', 'write', 'child', 'finish', 'production', 'currently', 'ship', 'packaging', 'arrive', 'excite']
['scorpio', 'aquarius', 'taurus']
['fucking', 'proud', 'metgala', 'pic.twitter.com/glvvp0pigz']
['anyone', 'least', 'times']
['dating', 'someone', 'understand', 'mental', 'health', 'important', 'humans', 'terrible', 'things', 'control']
['someone', 'wanna', 'solid', 'heart', 'thanks']
['finally', 'relationship', 'someone', 'actually', 'care', 'relationship', 'care', 'mental', 'health', 'support', 'dream']
['facebook', 'thought', 'need', 'share', 'pic.twitter

In [144]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [150]:
import gensim
NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=300)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.083*"things" + 0.083*"pic.twitter.com/jg3ayhs2k9" + 0.083*"explain" + 0.083*"anywhere"')
(1, '0.233*"bitch" + 0.083*"sensitive" + 0.008*"wanna" + 0.008*"actually"')
(2, '0.011*"friend" + 0.011*"hire" + 0.011*"thanks" + 0.011*"actually"')
(3, '0.040*"things" + 0.040*"humans" + 0.040*"control" + 0.040*"understand"')
(4, '0.049*"money" + 0.049*"split" + 0.049*"please" + 0.049*"place"')
(5, '0.077*"someone" + 0.077*"thanks" + 0.077*"heart" + 0.077*"solid"')
(6, '0.067*"least" + 0.067*"steve" + 0.067*"reunion" + 0.067*"dustin"')
(7, '0.103*"relationship" + 0.103*"care" + 0.054*"someone" + 0.054*"mental"')
(8, '0.043*"going" + 0.043*"someone" + 0.022*"start" + 0.022*"matter"')
(9, '0.067*"enjoy" + 0.067*"summer" + 0.067*"pic.twitter.com/egztopntuf" + 0.067*"canela"')


## Getting list of top subreddits

In [20]:
driver = webdriver.Chrome()
driver.get('https://www.reddit.com/r/TheoryOfReddit/comments/1f7hqc/the_200_most_active_subreddits_categorized_by/')

In [21]:
subs = driver.find_elements_by_class_name('_1qeIAgB0cPwnLhDF9XSiJM')

In [55]:
subreddits = []
for i in subs[3:]:
    if "r/" in i.text:
        subreddits.append(i.text.split(" - ")[1])
    else:
        continue

In [65]:
clean_subs = []
for sub in subreddits:
    clean_subs.append(sub.split('r/')[1])

## Using PRAW (Python Reddit API Wrapper) to get top posts in top subreddits

In [5]:
import praw
reddit = praw.Reddit(client_id='MU9yce7zmiOKwA',
                     client_secret='DRV5fO7p-Tn7VzxmMH8mnRwM_hE',
                     user_agent='my user agent')

In [68]:
all_submissions = {}
for sub in clean_subs:
    all_submissions[sub] = []
    for submission in reddit.subreddit(sub).top(limit=10):
        all_submissions[sub].append(submission.title)
    

In [69]:
all_submissions

{'AskReddit': ["People who haven't pooped in 2019 yet, why are you still holding on to last years shit?",
  'Stan Lee has passed away at 95 years old',
  'How would you feel about a feature where if someone upvotes a crosspost, the original post is upvoted automatically?',
  'Reddit, how would you feel about a law that bans radio stations from playing commercials with honking/beeping/siren noises in them?',
  'Without saying what the category is, what are your top five?',
  'Anthony Bourdain once said "There\'s a guy in my head, and all he wants to do is lay in bed all day long, smoke pot, and watch old movies and cartoons. My life is a series of strategems, to avoid, and outwit that guy". Who is "that guy" for you, and what do you do to avoid him?',
  'With all of the negative headlines dominating the news these days, it can be difficult to spot signs of progress. What makes you optimistic about the future?',
  '[Serious] Americans, would you be in support of putting a law in place th

In [17]:
for submission in reddit.subreddit('memes').top(limit=100):
    print(submission.title)

I drew all the boys together and i did it for the internet
Simpsons predicted it yet again
For real though
Hey Blizzard, I heard you like China's money ?
Actual Air Force Brief My Cousin Had to Sit Through
Its all coming together
Shut it off
And that's a fact
This is why Reddit is better
Could you not fall out of the buns FOR ONE SECOND!
The Area 51 raid is still happening right?
He did it
I bought a label maker for this joke
Guess who?
And so it begins
Quicksave complete
I'm getting nervous
Humans are much interest
Made with slow internet
*confused screaming*
You 1.8k people wanted it
Can we just appreciate everyone equally?
how do they know?
*dies inside for about 1 second*
Shame on them
One apple a year keeps your wallet empty
Took some time to find the HD clip of Peter dancing and the original image of The Joker to put this together. Now I shall rest.
Maybe if i become a dog i would be happy too
Wait a minute......
I'm going on an adventure
The whole of Reddit deserves the renaming

That happened to me today
best pun of the year
The people on reddit
High eating be like
I was so surprised
Please stop
My office vs Japanese office
GaMeS cAuSe ViOlEnCe
ALERT non Area 51 MEME incoming
It's all about tactics
This is the truest thing ever
I mean I don’t mind, just sayin’...
My disappointment is miserable
I knew it!
This is far beyond my possibilities
Big quacc
I’m the atom
Watermelon car is good
Mama, OwOwOwO
"We're not making that mistake again."
Deploy the rockets!!
All dogs are good boys
Karen and the crew
They know
Got a headache? Drill a hole in your head. Easy Peasy!
Just don't do it
You lucky boi
The "highest" level of design
Not the hero we deserved but the one we needed
Pro gamer move
Need a mental ad blocker
For real tho
It's an original name, I have to admit
harry potter is about to kick your ass
Am I a joke to you?
Day 1 of posting cursed images
Hell yeah
McFlurry
Reddit is superior
Weird flex
Remember that kid from stranger things?
Plot twist
A bullet can't 

This Would Be Me.
Can’t make a sentence with a
I dunno man
Ha! got em
Every dad
I like side missions
remastered xD
Carefully, he's a hero
It would explain a lot
This is fashion
Made this in my woodworking class.... got an 11/10.
Scout Typer
How could this have happened
Ahh that's hot, that's real hot
I know what to spend my time on.
this day extracts a heavy toll
Sexism bad Instagram Bad
Hey it's you
It do be like that
Microtransactions are a problem.
He makes a good point
Unrealistic
They asked for it
They are all breathtaking pups too
Good ol' Tom
They did it for you.
Gotta love it
Oh sorry son I had to work outside the country for you but now I am here
They always get us wrong.
Fresh hot n ready
He’s not bad, guys
Correct me if I'm wrong
I can relate
Big brain meme
I’m glad I missed it
They’re called memes and they’re funny
He does have a point
Fun fact:
Please don’t misunderstand this meme
Hmmmmmmm
Wholesome Girlfriend's Dad
Big brain time
A meme my teacher showed during her presen